### Etapas do projeto
- Entendimento e definição do problema e geração de hipóteses
- Aquisição dos dados
- Limpeza (remover null, preecher null, transformar dados categoricos, filtrar)
- Análise exploratória: Conhecer as distribuições, variância, corelação (remover correlações muito fortes)
- Manipulação dos dados: seleção, formatação, agregação, preparação de acordo com os modelos
- Modelagem geração de modelos: parametrização
- Avaliação: acurácia, taxa de erro, matriz de confusão
- Deploy

In [1]:
import pandas as pd
url_dados = "https://bi.s3.es.gov.br/covid19/MICRODADOS.csv"
dados = pd.read_csv(url_dados, sep=";", encoding="latin1", nrows=1_000_000)

In [2]:
# Schema
dados.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 45 columns):
 #   Column                   Non-Null Count    Dtype 
---  ------                   --------------    ----- 
 0   DataNotificacao          1000000 non-null  object
 1   DataCadastro             1000000 non-null  object
 2   DataDiagnostico          1000000 non-null  object
 3   DataColeta_RT_PCR        531916 non-null   object
 4   DataColetaTesteRapido    355611 non-null   object
 5   DataColetaSorologia      44183 non-null    object
 6   DataColetaSorologiaIGG   57725 non-null    object
 7   DataEncerramento         829190 non-null   object
 8   DataObito                8687 non-null     object
 9   Classificacao            1000000 non-null  object
 10  Evolucao                 1000000 non-null  object
 11  CriterioConfirmacao      1000000 non-null  object
 12  StatusNotificacao        1000000 non-null  object
 13  Municipio                1000000 non-null  object
 14  Bai

In [16]:
# Selecionando apenas as colunas object e descrevendo
dados.select_dtypes(include="object").describe().T[["unique", "top"]]

,unique,top
DataNotificacao,226,2021-03-22
DataCadastro,376,2021-03-24
DataDiagnostico,389,2021-03-15
DataColeta_RT_PCR,514,2021-03-22
DataColetaTesteRapido,456,2021-05-31
DataColetaSorologia,376,2020-11-23
DataColetaSorologiaIGG,383,2020-12-03
DataEncerramento,401,2021-04-05
DataObito,235,2021-04-19
Classificacao,3,Descartados


## Técnicas para transformar variaveis categoricas em numero
- Categóricas NOMINAIS (Os valores são object e não guardam relação de ordem)
- Categóricas ORDINAIS (São valores do tipo object mas guardam uma representação ordinal entre elas

### Variáveis Nominais (OneHotEncoder)
- one hot enconde (variaveis nominais) - Transforma uma coluna em várias outras, uma para cada opção da coluna original 

In [18]:
# !pip install category_encoders
from category_encoders.one_hot import OneHotEncoder

print(dados["Gestante"].unique())
# Passar as colunas que serão transformadas de texto para numeros
one_hot_enc = OneHotEncoder(cols=["Gestante"])

dados_categoricos = dados[['Escolaridade', 'Gestante','Febre', 'DificuldadeRespiratoria', 'Tosse', 'Coriza', 'DorGarganta', 'Diarreia', 'Cefaleia', 'ComorbidadePulmao', 'ComorbidadeCardio', 'ComorbidadeRenal', 'ComorbidadeDiabetes', 'ComorbidadeTabagismo', 'ComorbidadeObesidade', 'FicouInternado']]

# Transforma as colunas do OneHotEncoder que estão nas colunas categoricas (Gestante virou Gestante_1	Gestante_2	Gestante_3	Gestante_4)
dados_categoricos = one_hot_enc.fit_transform(dados_categoricos)


['Não' 'Não se aplica' '2º trimeste' '3º trimeste' 'Ignorado'
 '1º trimeste' 'Idade gestacional ignorada']


C:\Users\BARBIERI\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,Escolaridade,Gestante_1,Gestante_2,Gestante_3,Gestante_4,Gestante_5,Gestante_6,Gestante_7,Febre,DificuldadeRespiratoria,...,DorGarganta,Diarreia,Cefaleia,ComorbidadePulmao,ComorbidadeCardio,ComorbidadeRenal,ComorbidadeDiabetes,ComorbidadeTabagismo,ComorbidadeObesidade,FicouInternado
0,Ignorado,1,0,0,0,0,0,0,Sim,Não,...,Não,Sim,Não,Não,Não,Não,Não,Não,Não,Não Informado
1,Ignorado,1,0,0,0,0,0,0,Não,Não,...,Não,Sim,Não,Não,Sim,Não,Não,Não,Não,Não
2,Ignorado,0,1,0,0,0,0,0,Sim,Não,...,Não,Sim,Sim,Não,Não,Não,Não,Não,Não,Não Informado
3,Não se aplica,0,1,0,0,0,0,0,Sim,Não,...,Não,Sim,Não,Não,Não,Não,Não,Não,Não,Não Informado
4,Educação superior completa,1,0,0,0,0,0,0,Não,Não,...,Sim,Não,Sim,Não,Não,Não,Não,Não,Não,Não Informado
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,Analfabeto,0,1,0,0,0,0,0,Sim,Não,...,Sim,Não,Não,Sim,Não,Não,Não,Não,Não,Não
999996,Educação superior completa,1,0,0,0,0,0,0,Não,Não,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não
999997,Ensino fundamental completo (antigo ginásio ou...,1,0,0,0,0,0,0,Não,Não,...,Não,Não,Sim,Não,Não,Não,Não,Não,Não,Não
999998,Educação superior completa,1,0,0,0,0,0,0,Não,Não,...,Não,Sim,Sim,Não,Não,Não,Não,Não,Não,Não


### Variáveis Ordinárias (OrdinalEncoder)
- Mantem apenas uma coluna mas transforma todas as opções em valores
- https://contrib.scikit-learn.org/category_encoders/onehot.html <br>
__Existe um tipo de encoder chamado de Label encode que é aplicado a apenas uma variável, normalmente aquela que será usada pare predição__

In [19]:
# Troca as categorias por numeros
from category_encoders import OrdinalEncoder

ordinal_enc = OrdinalEncoder(cols=['Escolaridade', 'Febre', 'DificuldadeRespiratoria', 'Tosse', 'Coriza', 'DorGarganta', 'Diarreia', 'Cefaleia', 'ComorbidadePulmao', 'ComorbidadeCardio', 'ComorbidadeRenal', 'ComorbidadeDiabetes', 'ComorbidadeTabagismo', 'ComorbidadeObesidade'])

dados_categoricos = ordinal_enc.fit_transform(dados_categoricos)

C:\Users\BARBIERI\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [21]:
# Análise dos dados
pd.crosstab([dados["Classificacao"], dados["Evolucao"]], dados["FicouInternado"])

FicouInternado                         Ignorado     Não  Não Informado   Sim
Classificacao Evolucao                                                      
Confirmados   -                             150    3770           6215   373
              Cura                         8788  125613         176611  2278
              Ignorado                      421    1613           1222    93
              Óbito pelo COVID-19           179    1819           1886  3197
              Óbito por outras causas         4      34             31    25
Descartados   -                            2042   24602          52476   332
              Cura                            9     137            336     1
              Ignorado                     8741  159908         228773  1772
              Óbito por outras causas        70     555            369   484
Suspeito      -                            3029   52501          73316   565
              Cura                           52    1039            983    15
              Ignorado                      657   22814          29929   161
              Óbito por outras causas         1       6              2     1

In [8]:
# Correlação entre as features
# dados.corr().T

In [22]:
dados_categoricos.head()

,Escolaridade,Gestante_1,Gestante_2,Gestante_3,Gestante_4,Gestante_5,Gestante_6,Gestante_7,Febre,DificuldadeRespiratoria,...,DorGarganta,Diarreia,Cefaleia,ComorbidadePulmao,ComorbidadeCardio,ComorbidadeRenal,ComorbidadeDiabetes,ComorbidadeTabagismo,ComorbidadeObesidade,FicouInternado
0,1,1,0,0,0,0,0,0,1,1,...,1,1,1,1,1,1,1,1,1,Não Informado
1,1,1,0,0,0,0,0,0,2,1,...,1,1,1,1,2,1,1,1,1,Não
2,1,0,1,0,0,0,0,0,1,1,...,1,1,2,1,1,1,1,1,1,Não Informado
3,2,0,1,0,0,0,0,0,1,1,...,1,1,1,1,1,1,1,1,1,Não Informado
4,3,1,0,0,0,0,0,0,2,1,...,2,2,2,1,1,1,1,1,1,Não Informado


In [24]:
ficou = dados_categoricos["FicouInternado"] == "Sim" #dados.Gestante.value_counts()
ficou.value_counts()

False    990703
True       9297
Name: FicouInternado, dtype: int64

In [25]:
# Todos os dados categoricos menos o targe
x = dados_categoricos.drop("FicouInternado", axis=1)
y = dados_categoricos["FicouInternado"] == "Sim"

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size = 0.2, random_state=1000, stratify=y)

# Criar o modelo
modelo_rlogistica = LogisticRegression(max_iter=1000)
modelo_rlogistica.fit(x_treino, y_treino)

# previsao
# previsoes = modelo_rlogistica.predict(x_teste)

# Faz o predict e testa o acerto
modelo_rlogistica.score(x_teste, y_teste)

0.99055

In [35]:
len(dados_categoricos.columns)

22

In [56]:
novo_teste = pd.DataFrame([0, 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]).T

In [57]:
# Novo
modelo_rlogistica.predict(novo_teste)


array([False])